# **MCB Final Project** 
code written by Sarah Williams Dec 1, 2023

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA

# transition probabilities
lambda12 = 0.05
lambda13 = 0.05
lambda11 = 1 - lambda12 - lambda13

lambda21 = 0.05
lambda23 = 0.05
lambda22 = 1 - lambda21 - lambda23

lambda31 = 0.05
lambda32 = 0.05
lambda33 = 1 - lambda31 - lambda32

# time steps
tauMax = 100

# fitnesses
w11 = 1
w12 = 0.1
w13 = 0.1
w21 = 0.1
w22 = 1
w23 = 0.1
w31 = 0.1
w32 = 0.1
w33 = 1

transitionMatrix = np.array([[lambda11, lambda21, lambda31], 
                             [lambda12, lambda22, lambda32], 
                             [lambda13, lambda23, lambda33]]) 

fitnessMatrix = np.array([[w11, w12, w13],
                          [w21, w22, w23],
                          [w31, w32, w33]])

w1 = fitnessMatrix[0]
w2 = fitnessMatrix[1]
w3 = fitnessMatrix[2]

def getProbMatrix(transitionMatrix):
    '''Get probability vector from eigenvalues and eigenvectors of transition matrix'''
    eigValues, eigVectors = LA.eig(transitionMatrix) 
    probVector = np.zeros(3)
    eigVectorsT = np.transpose(eigVectors)

    # find eigenvalue that is = 1 and get the corresponding eigenvector
    for i in range(len(eigValues)):
        if (1-eigValues[i] < 0.000001):
            probVector = eigVectorsT[i, :]
    
    # normalize eig vector
    probSum = np.sum(probVector)
    for i in range(probVector.shape[0]):
        probVector[i] = probVector[i]/probSum

    # repeat vector to make 3x3 matrix
    probVectorMatrix = np.stack((probVector, probVector, probVector))
    
    return probVectorMatrix


def plasticFitness(transitionMatrix, fitnessMatrix, tauMax):
    '''Calculates plastic fitnesses over tau time steps'''
    fitnessL = []

    probMatrix = getProbMatrix(transitionMatrix)

    # iterate over values of tau 
    for tau in range(1, tauMax+1):
        tauMatrix = LA.matrix_power(transitionMatrix, tau) # transition matrix raised to tau 
        fitnessTauMatrix = np.multiply(tauMatrix, fitnessMatrix)
        plasticMatrix = np.multiply(fitnessTauMatrix, probMatrix)
        fitnessL.append(np.sum(plasticMatrix))
    return fitnessL

def fixedFitness(transitionMatrix, fitnessVector):
    '''Calculates fixed fitness of input fitness vector for comparison to plastic fitness'''
    probVector = getProbMatrix(transitionMatrix)[0]
    fixedFitness = np.sum(np.multiply(probVector, fitnessVector))

    return fixedFitness


fitnesses = plasticFitness(transitionMatrix, fitnessMatrix, tauMax)
w1Fixed = fixedFitness(transitionMatrix, w1)
w2Fixed = fixedFitness(transitionMatrix, w2)
w3Fixed = fixedFitness(transitionMatrix, w3)

w1L = [w1Fixed] * tauMax
w2L = [w2Fixed] * tauMax
w3L = [w3Fixed] * tauMax

plt.plot(range(tauMax), fitnesses, label="plastic")
plt.plot(range(tauMax), w1L, label="w1 fixed")
plt.plot(range(tauMax), w2L, label="w2 fixed")
plt.plot(range(tauMax), w3L, label="w3 fixed")
plt.xlabel('Tau')
plt.ylabel('Fitness')
plt.ylim(0,1)
plt.legend()
plt.show()

In [ ]:
import random
import matplotlib.pyplot as plt

# Environment switching agent-based simulation
# left out of final analysis because it does not work the way intended (envSwitch has a bug) 
# and doesn't contribute to research question

def startingPop(popsize):
    '''Create the starting distribution of popsize individuals in n environments'''
    individpop = int(popsize/3)
    pop = individpop*['1'] + individpop*['2'] + individpop*['3']
    random.shuffle(pop)

    return pop

lambda12 = 0.05
lambda13 = 0.05
lambda21 = 0.05
lambda23 = 0.05
lambda31 = 0.05
lambda32 = 0.05

def envSwitch(lambda12, lambda13, lambda21, lambda23, lambda31, lambda32, pop):
    '''Switches individuals between environments over one time step'''
    for i in range(len(pop)):
        rand = random.random()
        if pop[i] == '1':
            if rand < lambda12:
                pop[i] = '2'
                break
            elif rand < lambda13:
                pop[i] = '3'
                break
            
        elif pop[i] == '2':
            if rand < lambda21:
                pop[i] = '1'
                break
            elif rand < lambda23:
                pop[i] = '3'
                break

        elif pop[i] == '3':
            if rand < lambda31:
                pop[i] = '1'
                break
            elif rand < lambda32:
                pop[i] = '2'
                break

    return pop

def timeDependent(lambda12, lambda13, lambda21, lambda23, lambda31, lambda32, pop, steps):
    '''Performs the environment switch for each step'''
    count1L = []
    count2L = []
    count3L = []
    count1L.append(pop.count('1'))
    count2L.append(pop.count('2'))
    count3L.append(pop.count('3'))
    for i in range(steps):
        envSwitch(lambda12, lambda13, lambda21, lambda23, lambda31, lambda32, pop)
        count1L.append(pop.count('1'))
        count2L.append(pop.count('2'))
        count3L.append(pop.count('3'))

    return pop, count1L, count2L, count3L

pop = startingPop(99)
steps = 10
finalpop, count1, count2, count3 = timeDependent(lambda12, lambda13, lambda21, lambda23, lambda31, lambda32, pop, steps)
print("Number of individuals in environment 1:", count1[-1])
print("Number of individuals in environment 2:", count2[-1])
print("Number of individuals in environment 3:", count3[-1])


plt.plot(range(steps+1), count1, label="environment 1")
plt.plot(range(steps+1), count2, label="environment 2")
plt.plot(range(steps+1), count3, label="environment 3")
plt.xlabel('Days')
plt.ylabel('Number of individuals')
plt.ylim(0,100)
plt.legend()
plt.show()